In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import re

In [2]:
# Specify the directory containing CSV files
csv_directory_path = r"E:\Code\data_blog\blog_files\data"

# List all files in the directory
all_files = os.listdir(csv_directory_path)
specific_csv_files = [f for f in all_files if f.endswith('.csv')]

# Create a list of full file paths for the specific CSV files
specific_csv_paths = [os.path.join(csv_directory_path, f) for f in specific_csv_files]

In [3]:
df = pd.read_csv(specific_csv_paths[2])

In [4]:
df.columns

Index(['Day', 'Group', 'Food Name', 'Amount', 'Energy (kcal)', 'Alcohol (g)',
       'Caffeine (mg)', 'Water (g)', 'B1 (Thiamine) (mg)',
       'B2 (Riboflavin) (mg)', 'B3 (Niacin) (mg)',
       'B5 (Pantothenic Acid) (mg)', 'B6 (Pyridoxine) (mg)',
       'B12 (Cobalamin) (µg)', 'Folate (µg)', 'Vitamin A (µg)',
       'Vitamin C (mg)', 'Vitamin D (IU)', 'Vitamin E (mg)', 'Vitamin K (µg)',
       'Calcium (mg)', 'Copper (mg)', 'Iron (mg)', 'Magnesium (mg)',
       'Manganese (mg)', 'Phosphorus (mg)', 'Potassium (mg)', 'Selenium (µg)',
       'Sodium (mg)', 'Zinc (mg)', 'Carbs (g)', 'Fiber (g)', 'Starch (g)',
       'Sugars (g)', 'Added Sugars (g)', 'Net Carbs (g)', 'Fat (g)',
       'Cholesterol (mg)', 'Monounsaturated (g)', 'Polyunsaturated (g)',
       'Saturated (g)', 'Trans-Fats (g)', 'Omega-3 (g)', 'Omega-6 (g)',
       'Cystine (g)', 'Histidine (g)', 'Isoleucine (g)', 'Leucine (g)',
       'Lysine (g)', 'Methionine (g)', 'Phenylalanine (g)', 'Protein (g)',
       'Threonine (g)', 

In [5]:
# Function to transform a single column name
def transform_column_name(col_name):
    col_name = col_name.lower()
    col_name = re.sub(r' ', '_', col_name)
    col_name = re.sub(r'[^a-z0-9_]', '', col_name)
    if not re.match(r'^[a-z_]', col_name):
        col_name = f'_{col_name}'
    col_name = col_name[:128]
    return col_name

new_column_names = [transform_column_name(col) for col in df.columns]
df.columns = new_column_names

In [6]:
df.columns

Index(['day', 'group', 'food_name', 'amount', 'energy_kcal', 'alcohol_g',
       'caffeine_mg', 'water_g', 'b1_thiamine_mg', 'b2_riboflavin_mg',
       'b3_niacin_mg', 'b5_pantothenic_acid_mg', 'b6_pyridoxine_mg',
       'b12_cobalamin_g', 'folate_g', 'vitamin_a_g', 'vitamin_c_mg',
       'vitamin_d_iu', 'vitamin_e_mg', 'vitamin_k_g', 'calcium_mg',
       'copper_mg', 'iron_mg', 'magnesium_mg', 'manganese_mg', 'phosphorus_mg',
       'potassium_mg', 'selenium_g', 'sodium_mg', 'zinc_mg', 'carbs_g',
       'fiber_g', 'starch_g', 'sugars_g', 'added_sugars_g', 'net_carbs_g',
       'fat_g', 'cholesterol_mg', 'monounsaturated_g', 'polyunsaturated_g',
       'saturated_g', 'transfats_g', 'omega3_g', 'omega6_g', 'cystine_g',
       'histidine_g', 'isoleucine_g', 'leucine_g', 'lysine_g', 'methionine_g',
       'phenylalanine_g', 'protein_g', 'threonine_g', 'tryptophan_g',
       'tyrosine_g', 'valine_g', 'category'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 57 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   day                     675 non-null    object 
 1   group                   675 non-null    object 
 2   food_name               675 non-null    object 
 3   amount                  675 non-null    object 
 4   energy_kcal             653 non-null    float64
 5   alcohol_g               373 non-null    float64
 6   caffeine_mg             420 non-null    float64
 7   water_g                 465 non-null    float64
 8   b1_thiamine_mg          456 non-null    float64
 9   b2_riboflavin_mg        452 non-null    float64
 10  b3_niacin_mg            452 non-null    float64
 11  b5_pantothenic_acid_mg  401 non-null    float64
 12  b6_pyridoxine_mg        421 non-null    float64
 13  b12_cobalamin_g         394 non-null    float64
 14  folate_g                432 non-null    fl

In [12]:
from helpers.database_manager import DatabaseManager

ModuleNotFoundError: No module named 'blog_files'